In [ ]:
# 
# QR Code generator library (Python)
# 
# Copyright (c) Project Nayuki. (MIT License)
# https://www.nayuki.io/page/qr-code-generator-library
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
# - The above copyright notice and this permission notice shall be included in
#   all copies or substantial portions of the Software.
# - The Software is provided "as is", without warranty of any kind, express or
#   implied, including but not limited to the warranties of merchantability,
#   fitness for a particular purpose and noninfringement. In no event shall the
#   authors or copyright holders be liable for any claim, damages or other
#   liability, whether in an action of contract, tort or otherwise, arising from,
#   out of or in connection with the Software or the use or other dealings in the
#   Software.
# 

from __future__ import annotations
import collections, itertools, re
from collections.abc import Sequence
from typing import Callable, Dict, List, Optional, Tuple, Union
import matplotlib.pyplot as plt
import numpy as np
from colorsys import hsv_to_rgb, yiq_to_rgb
from PIL import Image, ImageOps
from colormath.color_objects import LabColor, AdobeRGBColor
from colormath.color_conversions import convert_color


class CQrCode:

	_version: int
	_size: int
	_modules: List[List[int]]
	_isfunction: List[List[int]]
	
	def __init__(self, version: int, color_space: str, file_path: str) -> None:
		if not (CQrCode.MIN_VERSION <= version <= CQrCode.MAX_VERSION):
			raise ValueError("Version value out of range")
		self._version = version
		self._size = version * 4 + 17
		self._color_space = color_space
		
		self._modules    = [[255, 255, 255] * self._size for _ in range(self._size)] 
		self._isfunction = [[255, 255, 255] * self._size for _ in range(self._size)]

		#Functional Modules and Grid derived from function outputs
		self._color_palette, self._colors = self._create_color_palette()
		self._modules_grid = self._modules_convert()
		self._isfunction_grid = self._isfunction_convert()

		#Writing the data and encoding
		self._data = self._read_data(file_path)
		self._encode_data()
	
	def get_version(self) -> int: #Getting the version of the CQRCode
		return self._version
	
	def get_size(self) -> int: #Getting the size which derives from the version
		return self._size
	
	def get_module(self, x: int, y: int) -> bool: #Getting the grid of modules
		return (0 <= x < self._size) and (0 <= y < self._size) and self._modules[y][x] == [0, 0, 0]

	def get_isfunction(self, x: int, y: int) -> bool: #Getting the modules which are functions
		return (0 <= x < self._size) and (0 <= y < self._size) and self._isfunction[y][x] == [0, 0, 0]
	
	def _read_data(self, file_path: str) -> None: #Openning the data from AAC file and reading it as a list of bytes
		with open(file_path, 'rb') as file:
			data = file.read()
		return [i for i in data]
	
	def _encode_data(self) -> None: #Gets the positions of non functional modules and fills in the data color in those areas
		#Get positions of non functional modules
		zero_positions = [(i, j) for i, row in enumerate(self._isfunction_grid) for j, value in enumerate(row) if value == 1]
		shortened = self._data[:len(zero_positions)] #Shorten the data to the length of the available spaces
		data_color_assignment = []
		for i in range(len(shortened)):
			index = self._data[i] #Index for the color encoding is the numerical number from data list
			data_color_assignment.append(self._colors[index]) #Assign the color to the non functional module
		for (row, col), color in zip(zero_positions, data_color_assignment): #Zipping the positions and the color together and then placing the color in the required modules
			self._modules_grid[row][col] = color
	
	def _create_color_palette(self) -> None: #Creating the color palette based on the input 
		if self._color_space == "yiq":
			colors = self._yiq()
		elif self._color_space == "hsv":
			colors = self._hsv()
		elif self._color_space == "lab":
			colors = self._lab()
		else:
			pass
		w = h = int(np.sqrt(len(colors)))
		full_palette = np.zeros((h+2, w+2, 3), dtype = np.uint8)
		color_matrix = np.array(colors).reshape(h, w, 3)
		full_palette[1:-1, 1:-1, :] = color_matrix
		return full_palette, colors

	def _yiq(self) -> List[List[int]]: #YIQ color space color palette
		y_levels = np.linspace(0.2, 0.6, 4)
		i_levels = np.linspace(-0.5, 0.5, 8)
		q_levels = np.linspace(-0.5, 0.5, 8)
		colors = []
		for i in i_levels:
			for q in q_levels:
				for y in y_levels:
					colors.append([int(i*255) for i in yiq_to_rgb(y, i, q)])
		return colors

	def _hsv(self) -> List[List[int]]: #HSV color space color palette
		hue = np.linspace(0, 1 - 1/16, 16)
		saturation = np.linspace(0.6, 1, 4)
		value = np.linspace(0.4, 1, 4)
		colors = []
		for h in hue:
			for v in value:
				for s in saturation:
					colors.append([int(i*255) for i in hsv_to_rgb(h,s,v)])
		return colors

	def _lab(self) -> List[List[int]]: #CIELAB color space color palette
		L = np.linspace(30, 50, 4)
		A = np.linspace(-128, 128, 8)
		B = np.linspace(-128, 128, 8)
		colors = []
		for b in B:
			for a in A:
				for l in L:
					lab_color = LabColor(l, a, b)
					rgb_color = convert_color(lab_color, AdobeRGBColor)
					rgb_values = [int(rgb_color.clamped_rgb_r *255),
									int(rgb_color.clamped_rgb_g*255), 
									int(rgb_color.clamped_rgb_b*255)]
					colors.append(rgb_values)
		return colors
	
	def _modules_convert(self) -> None: #Drawing all the functional patterns and then getting the modules as well, converting it to a grid of RGB values
		self._draw_function_patterns()
		self._draw_color_palette()
		qr_code_rgb = np.zeros((self._size, self._size, 3), dtype=np.uint8)
		for y in range(self._size):
			for x in range(self._size):
				qr_code_rgb[y,x] = self._modules[y][x]
		return qr_code_rgb
	
	def _isfunction_convert(self) -> None: #Converting to a list of 1 and 0 to signify if its a functional module or not, where 1 is true
		self._draw_function_patterns()
		self._draw_color_palette()
		qr_code_matrix = np.zeros((self._size, self._size))
		for y in range(self._size):
			for x in range(self._size):
				qr_code_matrix[y, x] = 0 if self.get_isfunction(x, y) else 1
		return qr_code_matrix
	
	def _draw_function_patterns(self) -> None: #Draws all functional patterns 
		for i in range(self._size): #Draws the functional modules alternating for timing patterns
			self._set_function_module(6, i, i % 2 == 0) #Vertical
			self._set_function_module(i, 6, i % 2 == 0) #Horizontal
		
		#Draw finder patterns at designated areas
		self._draw_finder_pattern(3, 3)
		self._draw_finder_pattern(self._size - 4, 3)
		self._draw_finder_pattern(3, self._size - 4) 
		
		# Draw alignment patters, skipping at certain areas
		alignpatpos: List[int] = self._get_alignment_pattern_positions()
		numalign: int = len(alignpatpos)
		skips: Sequence[Tuple[int,int]] = ((0, 0), (0, numalign - 1), (numalign - 1, 0), (numalign - 1, numalign - 1))
		for i in range(numalign):
			for j in range(numalign):
				if (i, j) not in skips: 
					self._draw_alignment_pattern(alignpatpos[i], alignpatpos[j])
		self._draw_color_palette() #Draws the color palette
	
	def _draw_finder_pattern(self, x: int, y: int) -> None: #Draw finder patterns 
		for dy in range(-4, 5):#-4, 5 used since center is given of a 9x9 grid, goes from -4 to 4
			for dx in range(-4, 5):
				xx, yy = x + dx, y + dy
				if (0 <= xx < self._size) and (0 <= yy < self._size):
					# Chebyshev/infinity norm
					self._set_function_module(xx, yy, max(abs(dx), abs(dy)) not in (2, 4)) #0, 1, 2, 3, 4 for the 9 x 9 grid
	
	def _draw_alignment_pattern(self, x: int, y: int) -> None: #Same method as the one above
		for dy in range(-2, 3):
			for dx in range(-2, 3):
				self._set_function_module(x + dx, y + dy, max(abs(dx), abs(dy)) != 1)

	def _draw_color_palette(self) -> None: #Draws color palette in its designated area
		palette_height, palette_width, _ = self._color_palette.shape
		border_size = 1  

		start_x = self._size - palette_width - border_size
		start_y = self._size - palette_height - border_size

		for dy in range(palette_height + border_size):
			for dx in range(palette_width + border_size):
				y_index = start_y + dy
				x_index = start_x + dx
				if dy == 0 or dx == 0:
					self._modules[y_index][x_index] = [255, 255, 255]  #For the border of the color palette
					self._isfunction[y_index][x_index] = [0, 0, 0] #Showing its a functional module
				else:
					self._modules[y_index][x_index] = self._color_palette[dy-1][dx-1].tolist() #For the rest, drawing the modules as colors from the color palette
					self._isfunction[y_index][x_index] = [0, 0, 0] #Showing its a functional module

        
	def _set_function_module(self, x: int, y: int, isdark: bool) -> None: #If true then draw as black, else white, but draw the whole area as a functional modules, dont wanna draw in there
		assert type(isdark) is bool
		if isdark:
			self._modules[y][x] = [0, 0, 0]
		else:
			self._modules[y][x] = [255, 255, 255]
		self._isfunction[y][x] = [0, 0, 0]
		
	def _get_alignment_pattern_positions(self) -> List[int]: #Drawing the alignment pattern positions, algorithm for the placement of positions, best works up until version 40 and then after it becomes a bit worse
		ver: int = self._version
		if ver == 1:
			return []
		else :
			numalign: int = ver // 7 + 2
			step: int = 26 if (ver == 32) else \
				(ver * 4 + numalign * 2 + 1) // (numalign * 2 - 2) * 2
			result: List[int] = [(self._size - 7 - i * step) for i in range(numalign - 1)] + [6]
			return list(reversed(result))
        
	def _show_cqr(self) -> None: #Draws modules grid on a figure
		fig = plt.figure(figsize = (16, 16), dpi=600)
		plt.imshow(self._modules_grid, interpolation='nearest')
		plt.axis("off")
		fig.savefig(f'Colored ({self._color_space}) QR Code v{version}.png', dpi=1200, bbox_inches='tight', pad_inches=0)

	def _show_functions(self) -> None: #Draws the functional module grids
		fig = plt.figure(figsize=(16, 16), dpi=600)
		plt.imshow(self._isfunction_grid, interpolation='nearest', cmap='gray')
		plt.axis("off")
		fig.savefig(f'Colored ({self._color_space}) QR Code v{version}_functions.png', dpi=1200, bbox_inches='tight', pad_inches=0)

	
	MIN_VERSION: int =  3  
	MAX_VERSION: int = 90 #Maximum version 90 as specified with a 25 ppm
	
	
file_path = "sample4.aac"
version = 3

cqr = CQrCode(version, "hsv",  file_path)
cqr._show_cqr()
cqr._show_functions()


In [ ]:
matrix = cqr._isfunction
print(matrix)

In [ ]:
capacity = 0
for i in range(len(matrix)):
    for j in range(len(matrix)):
        if matrix[i][j] != [0, 0, 0]:
            capacity += 1
        else:
            pass

print(f"{capacity} available modules")
print(f"Maximum of {capacity} Bytes of information")

bitrate = 64/8 #kB/s
seconds = capacity/(bitrate*1000)
print(f"With AAC files of bitrate {bitrate} kB/s, {seconds} seconds of audio can be stored")

In [ ]:
def get_image_size_r(ppm_value, dpi, version):
    w = h = 17 + version * 4
    ppmm_squared = (dpi / 25.4) ** 2
    pixels = w * h * (ppm_value**2)
    size_on_paper = pixels / ppmm_squared
    w_h = np.sqrt(size_on_paper)  # Convert size to mm for reportlab
    return w_h

n = get_image_size_r(4, 1200, version)
print(n)

In [ ]:
import matplotlib.pyplot as plt

ppml = [4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196]
size_max = 200

def capacity(w, ppm, dpi):
    """Calculate the capacity based on width, ppm (pixels per module), and dpi (dots per inch)."""
    size = w**2  # Total area in square millimeters
    ppm_squared = (dpi / 25.4) ** 2  # Convert dpi to dots per millimeter squared
    pixels = size * ppm_squared  # Total pixels
    modules = pixels / (ppm**2)  # Number of modules based on ppm
    return modules

cap = [capacity(size_max, i, 1200) for i in ppml]  # Calculate capacities for different ppml values
new_cap = [i/1000 for i in cap]  # Convert capacities to kilobytes

# Plotting
plt.figure(figsize=(15, 6))  # Wider aspect ratio
plt.plot(ppml, new_cap, marker='o', linestyle='-', color='b', label='Capacity vs PPM')
plt.xlabel('PPM', fontsize=14)
plt.ylabel('Theoretical Capacity (kB)', fontsize=14)

# Setting custom x-ticks at every 20 units starting from 0
x_ticks = range(0, max(ppml) + 1, 20)
plt.xticks(x_ticks)  # Apply custom x-ticks

plt.xlim(0, max(ppml) + 1)  # Adjust x-limits to start at 0 and end slightly beyond the max value for padding
plt.ylim(0, max(new_cap) * 1.1)  # Adjust y-limits to add some space above the highest data point

# Enable grid for both major and minor ticks; set major grid lines every 20 units
plt.grid(visible=True, which='both', linestyle='--', linewidth=0.5)
plt.minorticks_on()  # Enable minor ticks

# Setting minor grid lines with finer control
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='gray')

plt.tight_layout()  # Adjust layout to prevent overlap and ensure everything fits nicely
plt.show()

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from PIL import Image
import numpy as np

# Remove the pixel limit for PIL images
Image.MAX_IMAGE_PIXELS = None

# DPI and conversion factor
dpi = 1200 # pixels per mm squared

In [ ]:

def get_image_size(ppm_value, dpi, version): #Getting the size of the image if it were on real paper, taking into account how many modules across, the dpi and ppm
    w = h = 17 + version * 4
    ppmm_squared = (dpi / 25.4) ** 2
    pixels = w * h * (ppm_value)**2
    size_on_paper = pixels / ppmm_squared
    w_h = np.sqrt(size_on_paper) * mm #* mm is from reportlab, without this it gives the actual size but must be converted to be used on a pdf 
    return w_h

# Dimensions of an A4 page
page_width, page_height = A4 #getting the standard A4 page size

# Function to place images in PDFs
def create_pdf_for_color_space(color_space, ppm_value, version): #Creating a pdf with the image in the center
    pdf_path = f'Colored_QR_{color_space}_{ppm_value}_v{version}.pdf'
    c = canvas.Canvas(pdf_path, pagesize=A4)

    # Calculate image size
    image_size = get_image_size(ppm_value, dpi, version) #Getting image size

    # Position for the image: center of the page
    x = (page_width - image_size) / 2
    y = (page_height - image_size) / 2

    # Place the image
    image_path = f'Colored ({color_space}) QR Code v{version}.png'
    c.drawImage(image_path, x, y, width=image_size, height=image_size)

    # Finalize the PDF
    c.save()
    print(f'PDF created: {pdf_path}')

In [ ]:
create_pdf_for_color_space("hsv", 25, version) #Creating required pdf given the color space, ppm and version